In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from Creator_dataset import Creator_dataset
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (100, 100)

AUTOTUNE = tf.data.AUTOTUNE

creator_dataset = Creator_dataset(BATCH_SIZE) 
dataset_train, dataset_val, dataset_test = creator_dataset.create_dataset("occupancy", "occupancy", True)

train_dataset = dataset_train.prefetch(buffer_size=AUTOTUNE)
validation_dataset = dataset_val.prefetch(buffer_size=AUTOTUNE)
test_dataset = dataset_test.prefetch(buffer_size=AUTOTUNE)

image_batch = np.concatenate([x for x, y in dataset_train], axis=0)
label_batch = np.concatenate([y for x, y in dataset_train], axis=0)
image_batch_test = np.concatenate([x for x, y in dataset_test], axis=0)
label_batch_test = np.concatenate([y for x, y in dataset_test], axis=0)

--------------------
Dataset <built-in function dir>
occupancy	32090	1003 bt. x 32 bt.size: done
--------------------
Train		802 bt. x 32 bt.size: done
Val		100 bt. x 32 bt.size: done
Test		101 bt. x 32 bt.size: done


In [3]:
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')])


model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [ ]:
history = model.fit(image_batch, label_batch, epochs=20, 
                    validation_data=(image_batch_test, label_batch_test))


plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


Epoch 1/20


C:\Users\PC\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


802/802 [==============================] - 67s 84ms/step - loss: 0.1704 - accuracy: 0.9285 - val_loss: 0.0529 - val_accuracy: 0.9799
Epoch 2/20
802/802 [==============================] - 73s 91ms/step - loss: 0.0623 - accuracy: 0.9790 - val_loss: 0.0408 - val_accuracy: 0.9823
Epoch 3/20
802/802 [==============================] - 74s 92ms/step - loss: 0.0500 - accuracy: 0.9832 - val_loss: 0.0477 - val_accuracy: 0.9805
Epoch 4/20
802/802 [==============================] - 73s 91ms/step - loss: 0.0449 - accuracy: 0.9843 - val_loss: 0.0395 - val_accuracy: 0.9836
Epoch 5/20
802/802 [==============================] - 72s 90ms/step - loss: 0.0369 - accuracy: 0.9887 - val_loss: 0.0331 - val_accuracy: 0.9870
Epoch 6/20
802/802 [==============================] - 73s 91ms/step - loss: 0.0356 - accuracy: 0.9886 - val_loss: 0.0308 - val_accuracy: 0.9873
Epoch 7/20
802/802 [==============================] - 73s 91ms/step - loss: 0.0306 - accuracy: 0.9904 - val_loss: 0.0338 - val_accuracy: 0.9873
Epo

In [ ]:
test_loss, test_acc = model.evaluate(image_batch_test,  label_batch_test, verbose=2)
#model.save("models/CNN_occupancy")


In [ ]:
image_batch_val = np.concatenate([x for x, y in dataset_val], axis=0)
label_batch_val = np.concatenate([y for x, y in dataset_val], axis=0)

val_loss, val_acc = model.evaluate(image_batch_val,  label_batch_val, verbose=2)

predictions = model.predict(image_batch_val)


predlist = predictions.tolist()
predlist = [x[0] for x in predlist]

pred = pd.DataFrame({"pred": predlist,
              "lab": label_batch_val})

pred["pred"] = pred["pred"] > 0.5
pred["pred"] = pred["pred"].astype(int)

pd.crosstab(index=pred["lab"], columns=pred["pred"])


In [ ]:
plt.figure(figsize=(10, 10))
for i in range(64):
  ax = plt.subplot(8, 8, i + 1)
  plt.imshow(image_batch_val[i])
  plt.title(str(predictions[i] > 0.5))
  plt.axis("off")